<a href="https://colab.research.google.com/github/rakesh-choudhury/dump/blob/master/NLP_YouTube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pyldavis

In [0]:
!pip install lime

  Using cached https://files.pythonhosted.org/packages/85/5e/e91792f198bbc5a0d7d3055ad552bc4062942d27eaf75c3e2783cf64eae5/Pillow-5.4.1-cp36-cp36m-manylinux1_x86_64.whl
ERROR: streamlit 0.57.3 has requirement pillow>=6.2.0, but you'll have pillow 5.4.1 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 7.1.1
    Uninstalling Pillow-7.1.1:
      Successfully uninstalled Pillow-7.1.1


In [0]:
!pip install chart_studio

In [0]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import math
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.externals import joblib
import scipy.sparse
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import warnings; warnings.simplefilter('ignore')
import random
%matplotlib inline



plt.style.use('ggplot')
import seaborn as sns
%matplotlib notebook

import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline


from lime.lime_text import LimeTextExplainer
from tqdm import tqdm
import string
import random
import operator
import seaborn as sns
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from statistics import *
import concurrent.futures
import time
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
import warnings
import nltk
from sklearn.utils import shuffle


# spaCy based imports
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English



In [0]:
import cufflinks as cf
import plotly
import plotly.offline as py
import plotly.graph_objs as go

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
youtube = pd.read_csv('/content/drive/My Drive/Datasets/Final_YouTube_Dataset/YouTube.csv')

In [0]:
youtube.head(70)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,Country,Category,like_percentage,popular,tags_clean,sent_score
0,0,0,0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10.0,2017-11-10 17:00:03+00:00,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579.0,787425.0,43420.0,125882.0,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Cannada,Music_videos,94.773995,1,Eminem Walk On Water Aftermath Shady Interscop...,0.0
1,1,1,1,0dBIkQ4Mz1M,17.14.11,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23.0,2017-11-13 17:00:00+00:00,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651.0,127794.0,1688.0,13030.0,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,Cannada,Pranks,98.696344,0,plush bad unboxing unboxing fan mail idubbbztv...,0.5
2,2,2,2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23.0,2017-11-12 19:05:24+00:00,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434.0,146035.0,5339.0,8181.0,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,Cannada,Pranks,96.472974,1,racist superman rudy mancuso king bach racist ...,-0.6
3,3,3,3,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24.0,2017-11-12 18:01:41+00:00,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828.0,132239.0,1989.0,17518.0,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,Cannada,Youtube_Celebrities,98.518193,1,ryan higa higatv nigahiga i dare you idy rhpc ...,-0.3
4,4,4,4,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10.0,2017-11-09 11:04:14+00:00,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622.0,1634130.0,21082.0,85067.0,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,Cannada,Music_videos,98.726326,1,edsheeran ed sheeran acoustic live cover offic...,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,65,65,65,xfmqmmBTRXw,17.14.11,24 Oras November 13 2017,24 Oras News Today,22.0,2017-11-12 19:00:07+00:00,"24 Oras November 13 2017|""24 Oras""|""24 Oras Ne...",137210.0,381.0,105.0,156.0,https://i.ytimg.com/vi/xfmqmmBTRXw/default.jpg,False,False,False,Cannada,Shows_Reviews,78.395062,0,24 Oras November 13 2017 24 Oras 24 Oras News ...,0.0
66,66,66,66,PV8maMvPqhw,17.14.11,AMAZING Street Food in CHINA | RARELY SEEN St...,The Food Ranger,19.0,2017-11-11 17:00:03+00:00,"the food ranger|""trevor james""|""food ranger""|""...",325851.0,8998.0,185.0,1272.0,https://i.ytimg.com/vi/PV8maMvPqhw/default.jpg,False,False,False,Cannada,Food,97.985408,0,the food ranger trevor james food ranger food ...,0.9
67,67,68,68,3zjhB7NYQyg,17.14.11,Let's Play - Civic Doodle with Tim Gettys,LetsPlay,20.0,2017-11-11 15:00:00+00:00,"Achievement Hunter|""AH""|""video game""|""gaming""|...",274190.0,6303.0,266.0,368.0,https://i.ytimg.com/vi/3zjhB7NYQyg/default.jpg,False,False,False,Cannada,Video_Games,95.950677,0,Achievement Hunter AH video game gaming game g...,1.0
68,68,69,69,F5mzb086QM8,17.14.11,Secrets of Dairy: Larson Dairy Farm,The Secrets of Dairy,22.0,2017-11-09 22:15:58+00:00,"secrets of dairy|""larson dairy farms""|""publix""...",29234.0,37.0,65.0,62.0,https://i.ytimg.com/vi/F5mzb086QM8/default.jpg,False,False,False,Cannada,Shows_Reviews,36.274510,0,secrets of dairy larson dairy farms publix pla...,0.0


In [0]:
youtube.isnull().sum()

Unnamed: 0                0
Unnamed: 0.1              0
Unnamed: 0.1.1            0
video_id                  0
trending_date             0
title                     0
channel_title             0
category_id               0
publish_time              0
tags                      0
views                     0
likes                     0
dislikes                  0
comment_count             0
thumbnail_link            0
comments_disabled         0
ratings_disabled          0
video_error_or_removed    0
Country                   0
Category                  0
like_percentage           0
popular                   0
tags_clean                0
sent_score                0
dtype: int64

In [0]:
youtube = youtube.dropna()

In [0]:
youtube.to_csv('YouTube.csv')

In [0]:
!cp YouTube.csv "/content/drive/My Drive/Datasets/Final_YouTube_Dataset"

In [0]:
youtube.describe()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,category_id,views,likes,dislikes,comment_count,like_percentage,popular,sent_score
count,237785.000000,237785.000000,237785.000000,237785.000000,2.377850e+05,2.377850e+05,2.377850e+05,2.377850e+05,237785.000000,237785.000000,237785.00000
mean,118892.000000,120616.442139,120616.442139,20.528755,1.006113e+06,3.351927e+04,1.763528e+03,3.837915e+03,91.786679,0.117976,0.15733
std,68642.761217,69779.346197,69779.346197,7.008499,3.902957e+06,1.353115e+05,1.815864e+04,2.194087e+04,10.643798,0.322581,0.48013
min,0.000000,0.000000,0.000000,1.000000,2.230000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,-1.00000
25%,59446.000000,60082.000000,60082.000000,19.000000,5.420700e+04,8.870000e+02,5.300000e+01,1.210000e+02,89.947664,0.000000,0.00000
50%,118892.000000,120549.000000,120549.000000,24.000000,2.158460e+05,4.369000e+03,2.170000e+02,5.450000e+02,95.685456,0.000000,0.00000
75%,178338.000000,181185.000000,181185.000000,24.000000,7.118010e+05,1.941900e+04,7.920000e+02,2.055000e+03,98.042056,0.000000,0.50000
max,237784.000000,241119.000000,241119.000000,44.000000,2.252119e+08,5.613827e+06,1.674420e+06,1.361580e+06,100.000000,1.000000,1.00000


In [0]:
all_tags = youtube['tags']
all_sent_values = []
all_sentiments = []

In [0]:
analyser = SentimentIntensityAnalyzer()
result = analyser.polarity_scores('paragraph')

In [0]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
def sentiment_value(paragraph):
    analyser = SentimentIntensityAnalyzer()
    result = analyser.polarity_scores(paragraph)
    score = result['compound']
    return round(score,1)

In [0]:
youtube.shape

(237785, 21)

In [0]:
len(youtube.tags.values)

237785

In [0]:
import re
youtube['tags_clean'] = 'Nan'
x=0
for i in youtube.tags.values:
  youtube['tags_clean'][x] = re.sub('[^A-Za-z0-9]+', ' ', i)
  x+=1
# re.sub('[^A-Za-z0-9]+', ' ', string)

In [0]:
youtube['tags_clean']

0         Eminem Walk On Water Aftermath Shady Interscop...
1         plush bad unboxing unboxing fan mail idubbbztv...
2         racist superman rudy mancuso king bach racist ...
3         ryan higa higatv nigahiga i dare you idy rhpc ...
4         edsheeran ed sheeran acoustic live cover offic...
                                ...                        
237780    aarons animals aarons animals cat cats kitten ...
237781                                                none 
237782    I gave safiya nygaard a perfect hair makeover ...
237783    Black Panther HISHE Marvel Infinity War How It...
237784             call of duty cod activision Black Ops 4 
Name: tags_clean, Length: 237785, dtype: object

In [0]:
len(youtube.tags_clean.values)

237785

In [0]:
youtube.head(2)

,Unnamed: 0,Unnamed: 0.1,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,Country,Category,like_percentage,popular,tags_clean,sent_score
0,0,0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10.0,2017-11-10 17:00:03+00:00,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579.0,787425.0,43420.0,125882.0,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Cannada,Music_videos,94.773995,1,Eminem Walk On Water Aftermath Shady Interscop...,0
1,1,1,0dBIkQ4Mz1M,17.14.11,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23.0,2017-11-13 17:00:00+00:00,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651.0,127794.0,1688.0,13030.0,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,Cannada,Pranks,98.696344,0,plush bad unboxing unboxing fan mail idubbbztv...,0.5


In [0]:
youtube['sent_score'] = 'Nan'
y = 0
for i in range(0,len(youtube.tags_clean.values)):
  youtube['sent_score'][y] = sentiment_value(youtube['tags_clean'][i])
  y += 1

In [0]:
youtube.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'video_id',
       'trending_date', 'title', 'channel_title', 'category_id',
       'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count',
       'thumbnail_link', 'comments_disabled', 'ratings_disabled',
       'video_error_or_removed', 'Country', 'Category', 'like_percentage',
       'popular', 'tags_clean', 'sent_score'],
      dtype='object')

In [0]:
sent_plot['sent_score'].groupby('channel')

0.08381248010394869

In [0]:
cat.columns

Index(['sent_score'], dtype='object')

In [0]:
youtube.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'video_id',
       'trending_date', 'title', 'channel_title', 'category_id',
       'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count',
       'thumbnail_link', 'comments_disabled', 'ratings_disabled',
       'video_error_or_removed', 'Country', 'Category', 'like_percentage',
       'popular', 'tags_clean', 'sent_score'],
      dtype='object')

In [0]:
youtube.groupby("channel_title").sum().sort_values(by = 'sent_score', ascending = False)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,category_id,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,like_percentage,popular,sent_score
channel_title,,,,,,,,,,,,,,
The Late Show with Stephen Colbert,94326615,95691007,95691007,18664.0,850246854.0,11491461.0,624228.0,1369834.0,0.0,0.0,0.0,74548.433515,118,786.0
The Tonight Show Starring Jimmy Fallon,63599372,64522268,64522268,11385.0,746834718.0,19266674.0,502775.0,1310100.0,0.0,0.0,0.0,47689.206222,142,476.5
Elhiwar Ettounsi,56959858,57766229,57766229,14064.0,66530891.0,359299.0,67454.0,97598.0,0.0,0.0,0.0,48811.240721,0,460.8
The Daily Show with Trevor Noah,55930740,56735585,56735585,9867.0,387124090.0,7518906.0,493498.0,1078890.0,0.0,0.0,0.0,41134.782451,25,402.5
SAB TV,39429431,40002843,40002843,9810.0,118709694.0,655685.0,99492.0,62805.0,0.0,0.0,0.0,35540.025973,2,385.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Les Grosses Têtes,23380490,23721356,23721356,4708.0,993042.0,6722.0,828.0,662.0,0.0,0.0,0.0,18924.090070,0,-120.6
Auf Streife,18008614,18250468,18250468,4314.0,15789479.0,347572.0,16605.0,56838.0,0.0,0.0,0.0,17349.008925,0,-121.4
Philip DeFranco,24136731,24455320,24455320,8209.0,412203366.0,21804068.0,569016.0,3935580.0,0.0,0.0,0.0,32749.496766,45,-170.4


In [0]:
len(youtube.channel_title.unique())

16862

In [0]:
youtube.dtypes

Unnamed: 0                  int64
Unnamed: 0.1                int64
Unnamed: 0.1.1              int64
video_id                   object
trending_date              object
title                      object
channel_title              object
category_id               float64
publish_time               object
tags                       object
views                     float64
likes                     float64
dislikes                  float64
comment_count             float64
thumbnail_link             object
comments_disabled            bool
ratings_disabled             bool
video_error_or_removed       bool
Country                    object
Category                   object
like_percentage           float64
popular                     int64
tags_clean                 object
sent_score                float64
dtype: object

In [0]:
!pip install streamlit

     |████████████████████████████████| 7.1MB 2.6MB/s 
     |████████████████████████████████| 522kB 46.9MB/s 
     |████████████████████████████████| 112kB 44.2MB/s 
     |████████████████████████████████| 5.3MB 42.5MB/s 
     |████████████████████████████████| 102kB 11.8MB/s 
     |████████████████████████████████| 2.1MB 33.5MB/s 
     |████████████████████████████████| 122kB 42.8MB/s 
  Created wheel for tornado: filename=tornado-5.1.1-cp36-cp36m-linux_x86_64.whl size=462323 sha256=4cb4fa1bc8f28463343f06630cbbe2f872badc43e16703f3bbd19603b88bbd9e
  Stored in directory: /root/.cache/pip/wheels/6d/e1/ce/f4ee2fa420cc6b940123c64992b81047816d0a9fad6b879325
  Created wheel for blinker: filename=blinker-1.4-cp36-none-any.whl size=13449 sha256=f3a58153fb2ff15765da3c0268237130632a5e88251de785437add842065d7da
  Stored in directory: /root/.cache/pip/wheels/92/a0/00/8690a57883956a301d91cf4ec999cc0b258b01e3f548f86e89
  Created wheel for watchdog: filename=watchdog-0.10.2-cp36-none-any.whl size=73

In [0]:
# How to run a Dash app in Google Colab

### Install ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2020-04-20 13:33:43--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 35.169.40.78, 34.226.171.201, 52.87.72.17, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.169.40.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  62.1MB/s    in 0.2s    

2020-04-20 13:33:44 (62.1 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
get_ipython().system_raw('./ngrok http 8501 &')

### Get the public URL where you can access the Dash app. Copy this URL.
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://e91c2623.ngrok.io


In [0]:
#!streamlit run RDM_visualise.py
!streamlit run StreamlitNLP.py


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.185.52.180:8501

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt
/usr/local/lib/python3.6/dist-packages/plotly/expres